<a href="https://colab.research.google.com/github/ftempesta/Data-Science-Online/blob/master/Tutorial_5_Privacidad_Diferencial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preámbulo

In [ ]:
!wget https://github.com/uvm-plaid/programming-dp/raw/master/notebooks/adult_with_pii.csv

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

adult = pd.read_csv('adult_with_pii.csv')
adult.head()

**Qué atributos del dataset son:**

- Identificadores o PII (*personally identifiable information*)
- No identificadores
- Cuasi-identificadores

¿Cómo des-identificamos la información? Eliminando las columnas!

In [ ]:
to_remove = ['Name', 'SSN']
adult_pii = adult[['Name', 'SSN', 'DOB', 'Zip']]

adult_data = adult.copy().drop(columns=to_remove)
adult_data.head()

Tras esta acción, ¿podemos *re-identificar* datos?

Supongamos que conocemos a una persona, llamada Karrie Trusslove, y sabemos algunos de sus datos, como su ZIP code (o código postal) y su fecha de nacimiento.

In [ ]:
karries_row = adult_pii[adult_pii['Name'] == 'Karrie Trusslove']
known_cols = []

pd.merge(karries_row, adult_data, left_on=known_cols, right_on=known_cols)

**¿A cuántas personas podemos re-identificar?**

In [ ]:
adult_pii['DOB'].value_counts() .hist()
plt.xlabel('Fechas de nacimiento')
plt.ylabel('Frecuencia');

**¿Qué podemos hacer?**

- Podemos hacer agregaciones y sólo divulgar información agregada. ¿Qué problema tiene esto?

- Podemos aplicar medidas sobre los datos, como k-anonimity, o differential privacy.

# Privacidad Diferencial

Decimos que una función (también llamada *mecanismo*) $F$ es $\epsilon$-diferencialmente privado si:

$$\frac{\Pr[F(D) = S]}{\Pr[F(D') = S]} \leq e^\epsilon$$

Donde $F$ aplica a un dataset $D$ o $D'$.

Esto nos dice que si modificamos un poco el dataset, y que al hacerlo, la función retorna el mismo resultado con probabilidad $e^\epsilon$, entonces la función es $\epsilon$ privada.

Por ejemplo, una función $F$ puede ser simplemente contar bajo una condición, como veremos a continuación.

**¿Cuántas personas tienen más de 40 años?**

In [ ]:
# con .shape[0] obtenemos el número de filas del data frame resultante

adult.loc[adult['Age'] >= 40].shape[0]

# Mecanismo de Laplace

La siguiente función satisface $\epsilon$-differential privacy:

$$F(x) = f(x) + \text{Laplace}(\frac{s}{\epsilon})$$

Donde $\text{Laplace}(S)$ retorna un valor al azar de una distribución de Laplace con centro 0 y escala $S$.

$s$ se conoce como la *sensitividad*. El valor $s=1$ es suficiente cuando queremos hacer consultas de conteo.

Usaremos $\epsilon=0.1$ y $s=1$ y consultamos nuevamente:

In [ ]:
sensitivity = 1
epsilon = 0.1

adult[adult['Age'] >= 40].shape[0] + np.random.laplace(loc=0, scale=sensitivity/epsilon)

Varíe el valor de `epsilon` y observe cómo cambian los resultados.

Ahora no podemos saber exactamente cuántas personas tienen edad mayor o igual a 40 años.


**¿Qué pasa si hacemos una consulta de conteo tal que sólo se obtenga 1 resultado?**

In [ ]:
karries_row = adult[adult['Name'] == 'Karrie Trusslove']
karries_row[karries_row['Target'] == '<=50K'].shape[0]

Ahora ya sabemos cómo "proteger" este tipo de consultas usando un mecanismo de Laplace:

In [ ]:
sensitivity = 1
epsilon = 0.1

karries_row = adult[adult['Name'] == 'Karrie Trusslove']
karries_row[karries_row['Target'] == '<=50K'].shape[0] + \
  np.random.laplace(loc=0, scale=sensitivity/epsilon)

# Privacidad diferencial "aproximada"

También es llamada $(\epsilon,\delta)$-differential privacy. Un mecanismo $F$ satisface esta definición si:

$$\Pr[F(D) = S] \leq e^\epsilon\Pr[F(D') = S] + \delta$$

Es decir, estamos permitiendo que con una probabilidad $\delta$ no se cumpla la definición de $\epsilon$-differential privacy, y sí con probabilidad $1-\delta$.

Generalmente uno utiliza un $\delta$ muy pequeño, como $1/N^2$, donde $N$ es el tamaño del dataset. 

# Mecanismo Gaussiano

El mecanismo Gaussiano añade "ruido normal" a los datos. Este mecanismo no satisface $\epsilon$-differential privacy, pero sí $(\epsilon,\delta)$-differential privacy.

$$F(x) = f(x) + \mathcal{N}(\sigma^2)$$

donde $$\sigma^2 = \frac{2s^2\log(1.25/\delta)}{\epsilon^2}$$

y $s$ es la "sensibilidad" o "sensitividad" y $\mathcal{N}(\sigma^2)$ denota una distribución normal con media 0 y varianza $\sigma^2$

Veamos cómo se comportan los mecanismos vistos hasta ahora, usando $\epsilon=1$ y $\delta=10^{-5}$.

In [ ]:
epsilon = 1
vals_laplace = [np.random.laplace(loc=0, scale=1/epsilon) for x in range(100000)]

delta = 10e-5
sigma = np.sqrt(2 * np.log(1.25 / delta)) * 1 / epsilon
vals_gauss = [np.random.normal(loc=0, scale=sigma) for x in range(100000)]

plt.hist(vals_laplace, bins=50, label='Laplace')
plt.hist(vals_gauss, bins=50, alpha=.7, label='Gaussian');
plt.legend();


Viendo las dos distribuciones. ¿Cómo se comparan? ¿En qué casos podría usar un mecanismo u el otro?

# Formas de uso

- Entregar una interfaz de acceso a los datos con mecanismos, y un "budget" de cantidad de consultas a realizar. (¿Por qué es necesario un budget?)

- Aplicar mecanismos a los datos y liberar el dataset con privacidad diferencial.

# Clasificación con DP

Usaremos la implementación de IBM https://github.com/IBM/differential-privacy-library

In [ ]:
!pip install diffprivlib

## Cargar datos

In [ ]:
import numpy as np
import sklearn as sk
from sklearn.pipeline import Pipeline

from diffprivlib import models

In [ ]:
X_train = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
                        usecols=(0, 4, 10, 11, 12), delimiter=", ")
y_train = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
                        usecols=14, dtype=str, delimiter=", ")


X_test = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
                        usecols=(0, 4, 10, 11, 12), delimiter=", ", skiprows=1)

y_test = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
                        usecols=14, dtype=str, delimiter=", ", skiprows=1)

# cada valor tiene un punto "." al final, por lo que se lo sacamos
y_test = np.array([a[:-1] for a in y_test])

## Pipeline sin DP

Aplicamos pre-procesamiento a los datos y luego entrenamos una regresión logística estándar:

In [ ]:
# sin DP:

pipe = Pipeline([
    ('scaler', sk.preprocessing.StandardScaler()),
    ('pca', sk.decomposition.PCA(2)),
    ('lr', sk.linear_model.LogisticRegression(solver="lbfgs"))
])

pipe.fit(X_train, y_train)
baseline = pipe.score(X_test, y_test)
print("Accuracy en test-set, sin DP:", (baseline))

## Pipeline con DP

La librería es compatible con sklearn, por lo que la interfaz es muy similar.

Los hiperparámetros fueron elegidos a mano abajo, pero deben ser seleccionados idealmente en base a información no-privada del dataset (ver [ejemplos](https://github.com/IBM/differential-privacy-library/tree/main/notebooks) de la librería)

In [ ]:
# con DP

dp_pipe = Pipeline([
    ('scaler', models.StandardScaler(bounds=([17, 1, 0, 0, 1], [90, 160, 10000, 4356, 99]))),
    ('pca', models.PCA(2, data_norm=5, centered=True)),
    ('lr', models.LogisticRegression(data_norm=5))
])

dp_pipe.fit(X_train, y_train)
print("Accuracy en el test-set, con DP (epsilon=3):", (dp_pipe.score(X_test, y_test)))

In [ ]:
epsilons = np.logspace(-3, 0, 500)
pipe_accuracy = []

for epsilon in epsilons:
    _eps = epsilon / 3
    dp_pipe.set_params(scaler__epsilon=_eps, pca__epsilon=_eps, lr__epsilon=_eps)
    
    dp_pipe.fit(X_train, y_train)
    pipe_accuracy.append(dp_pipe.score(X_test, y_test))

In [ ]:
plt.semilogx(epsilons, pipe_accuracy, label="Pipeline con DP", zorder=10)
plt.plot(epsilons, np.ones_like(epsilons) * baseline, dashes=[2,2], label="Pipeline sin DP", zorder=5)
plt.title("Accuracy con DP")
plt.xlabel("epsilon")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.xlim(epsilons[0], epsilons[-1])
plt.legend(loc=4)
plt.show()

Podemos observar que a partir de $\epsilon=0.1$ el accuracy comienza a estabilizarse alrededor del accuracy obtenido con los datos originales.

# Referencias

- Programming Differential Privacy book: https://uvm-plaid.github.io/programming-dp/intro.html
- Differential Privacy library: https://github.com/IBM/differential-privacy-library/tree/main/notebooks
- https://desfontain.es/privacy/k-anonymity.html